In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/home/user/wylee/ kogpt_new/datasets/KETI_nopreprocessed_labeled_dropdup+seperate_ver0.3.csv')

In [3]:
df = df[['transcription', 'OCEAN']]
print(df.head())

                   transcription  \
0  저희가 지금 봐야 되는 게 설치 장소랑 뭘 팔지네요.   
1            그러니까 계절이 10월이니까 쌀쌀한   
2                          좀 쌀쌀한   
3         날씨 그럼 뭐 각자 좀 생각을 해볼까요?   
4      아니면 그냥 저희끼리 얘기를 하는 게 편하신지   

                                               OCEAN  
0  [0.39583333333333337, 0.7708333333333333, 0.72...  
1  [0.39583333333333337, 0.7708333333333333, 0.72...  
2  [0.7708333333333333, 0.5416666666666666, 0.770...  
3  [0.39583333333333337, 0.7708333333333333, 0.72...  
4  [0.39583333333333337, 0.7708333333333333, 0.72...  


In [4]:
import ast 

# OCEAN 컬럼을 리스트로 변환 후 소숫점 4자리로 반올림
def round_ocean_values(ocean_string):
    ocean_list = ast.literal_eval(ocean_string)  # 문자열을 리스트로 변환
    return [round(value, 4) for value in ocean_list]  # 각 값을 소숫점 4자리로 반올림

In [5]:
import ast 

# OCEAN 컬럼을 리스트로 변환 후 소숫점 4자리로 반올림
def round_ocean_values(ocean_string):
    ocean_list = ast.literal_eval(ocean_string)  # 문자열을 리스트로 변환
    return ocean_list  # 각 값을 소숫점 4자리로 반올림

# OCEAN 값 처리
df['OCEAN'] = df['OCEAN'].apply(round_ocean_values)
df['OCEAN'] = df['OCEAN'].apply(lambda lst: ', '.join([str(num) for num in lst]))
# print(type(df['OCEAN'][0]))
print(type(df['OCEAN'][0][0]))
print(type(df['OCEAN'][0]))
print(type(df['OCEAN']))
# 확인
print(df[['transcription', 'OCEAN']].head())

<class 'str'>
<class 'str'>
<class 'pandas.core.series.Series'>
                   transcription  \
0  저희가 지금 봐야 되는 게 설치 장소랑 뭘 팔지네요.   
1            그러니까 계절이 10월이니까 쌀쌀한   
2                          좀 쌀쌀한   
3         날씨 그럼 뭐 각자 좀 생각을 해볼까요?   
4      아니면 그냥 저희끼리 얘기를 하는 게 편하신지   

                                               OCEAN  
0  0.39583333333333337, 0.7708333333333333, 0.729...  
1  0.39583333333333337, 0.7708333333333333, 0.729...  
2  0.7708333333333333, 0.5416666666666666, 0.7708...  
3  0.39583333333333337, 0.7708333333333333, 0.729...  
4  0.39583333333333337, 0.7708333333333333, 0.729...  


In [6]:
instruction = "주어진 대화를 보고, 성격 5요인을 예측하세요."

# Instruction + Transcription 결합
df['input_text'] = df['transcription'].apply(lambda x: f"Instruction: {instruction} Transcription: {x}")

# 필요 없는 컬럼 제거
df = df[['input_text', 'OCEAN']]

print(df.head())  # 데이터 확인

                                          input_text  \
0  Instruction: 주어진 대화를 보고, 성격 5요인을 예측하세요. Transc...   
1  Instruction: 주어진 대화를 보고, 성격 5요인을 예측하세요. Transc...   
2  Instruction: 주어진 대화를 보고, 성격 5요인을 예측하세요. Transc...   
3  Instruction: 주어진 대화를 보고, 성격 5요인을 예측하세요. Transc...   
4  Instruction: 주어진 대화를 보고, 성격 5요인을 예측하세요. Transc...   

                                               OCEAN  
0  0.39583333333333337, 0.7708333333333333, 0.729...  
1  0.39583333333333337, 0.7708333333333333, 0.729...  
2  0.7708333333333333, 0.5416666666666666, 0.7708...  
3  0.39583333333333337, 0.7708333333333333, 0.729...  
4  0.39583333333333337, 0.7708333333333333, 0.729...  


In [7]:
from datasets import Dataset

# Hugging Face Dataset으로 변환
dataset = Dataset.from_pandas(df)
dataset = dataset.shuffle(seed=42)  # 데이터 섞기
dataset = dataset.train_test_split(test_size=0.2)  # 80% 학습, 20% 검증

# 데이터 확인
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['input_text', 'OCEAN'],
        num_rows: 9045
    })
    test: Dataset({
        features: ['input_text', 'OCEAN'],
        num_rows: 2262
    })
})


In [8]:
import torch
import torch.nn as nn
from transformers import GPT2PreTrainedModel, GPT2Model

class KoGPT2ForRegression(GPT2PreTrainedModel):
    """
    KoGPT2를 Regression Task(OCEAN 예측) 가능하도록 수정한 모델.
    """
    def __init__(self, config):
        super().__init__(config)
        self.transformer = GPT2Model(config)  # 기존 KoGPT2 구조 유지
        self.regression_head = nn.Linear(config.hidden_size, 5)  # 🔹 768 → 5로 변경

        self.init_weights()

    def forward(self, input_ids, attention_mask=None, labels=None):
        """
        Forward pass
        """
        outputs = self.transformer(input_ids, attention_mask=attention_mask)
        hidden_states = outputs.last_hidden_state[:, -1, :]  # 🔹 마지막 토큰의 hidden state 사용
        logits = self.regression_head(hidden_states)  # 🔹 Linear Layer 통과
        
        loss = None
        if labels is not None:
            loss_fn = nn.SmoothL1Loss()  # 🔹 Regression을 위한 MSE Loss 사용
            loss = loss_fn(logits, labels)  # MSE 계산
        
        return {"loss": loss, "logits": logits}

2025-03-30 20:03:22.561101: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743332602.572108 3046861 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743332602.575502 3046861 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-30 20:03:22.588464: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model = KoGPT2ForRegression.from_pretrained('skt/kogpt2-base-v2')
tokenizer = AutoTokenizer.from_pretrained(
    'skt/kogpt2-base-v2',
    bos_token='</s>', eos_token='</s>', unk_token='<unk>', pad_token='<pad>',
    padding_side="right",
    model_max_length=512,
)

Some weights of KoGPT2ForRegression were not initialized from the model checkpoint at skt/kogpt2-base-v2 and are newly initialized: ['regression_head.bias', 'regression_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
print(model)

KoGPT2ForRegression(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (regression_head): Linear(in_features=768, out_features=5, bias=True)
)


In [11]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from transformers import  pipeline
from transformers import Trainer, TrainingArguments
from copy import deepcopy
from dataclasses import dataclass

In [12]:
import torch
from torch.utils.data import Dataset
import pandas as pd
import transformers
import copy
import logging
from typing import Optional, Dict, Sequence

class SFT_dataset(Dataset):
    def __init__(self, df: pd.DataFrame, tokenizer: transformers.PreTrainedTokenizer, verbose=False):
        logging.warning("Loading data...")

        # 프롬프트 템플릿 (Transcription을 입력으로 사용)
        PROMPT_DICT = {
            "prompt_input": (
                "### Instruction(명령어):\n주어진 대화를 보고, 성격 5요인을 예측하세요.\n\n"
                "### Transcription(대화):\n{transcription}\n\n"
                "### Response(응답):"
            )
        }

        # 모델 입력 데이터 생성 (transcription을 입력으로)
        sources = []
        for _, row in df.iterrows():
            prompt = PROMPT_DICT["prompt_input"].format(
                transcription=row["input_text"]  # 대화 입력
            )
            sources.append(prompt)

        # 정답 데이터 (OCEAN 값) 생성 (string → tensor 변환)
        targets = []
        for _, row in df.iterrows():
            ocean_values = eval(row['OCEAN'])  # 🔹 문자열을 리스트로 변환
            targets.append(torch.tensor(ocean_values, dtype=torch.float32))  # 🔹 Float Tensor로 변환

        # 토큰화 (프롬프트만)
        sources_tokenized = self._tokenize_fn(sources, tokenizer)  # instruction + transcription만

        # 토큰화된 input_ids 저장
        self.input_ids = sources_tokenized["input_ids"]
        
        # labels에 직접 OCEAN float tensor 저장 (MSELoss 적용을 위해)
        self.labels = targets  

        logging.warning("Loading data done!!: %d" % len(self.labels))

    def _tokenize_fn(self, strings: Sequence[str], tokenizer: transformers.PreTrainedTokenizer) -> Dict:
        tokenized_list = tokenizer(
            strings,
            return_tensors="pt",
            padding="longest",
            max_length=tokenizer.model_max_length,
            truncation=True,
        )
        input_ids = tokenized_list["input_ids"]
        
        return dict(
            input_ids=input_ids
        )

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, i) -> Dict[str, torch.Tensor]:
        return dict(input_ids=self.input_ids[i], labels=self.labels[i])  # 🔹 labels을 float tensor로 반환


In [13]:
from dataclasses import dataclass
from typing import Dict, Sequence
import torch
import transformers

@dataclass
class DataCollatorForSupervisedDataset:
    tokenizer: transformers.PreTrainedTokenizer

    def __call__(self, instances: Sequence[Dict]) -> Dict[str, torch.Tensor]:
        """
        Regression Task에 맞춰 labels를 float tensor로 변환하고, 
        적절한 padding을 적용하는 collator.
        """
        input_ids, labels = tuple([instance[key] for instance in instances] for key in ("input_ids", "labels"))
        
        # 🔹 input_ids 패딩 (기존 방식 유지)
        input_ids = torch.nn.utils.rnn.pad_sequence(
            input_ids, batch_first=True, padding_value=self.tokenizer.pad_token_id
        )
        
        # 🔹 labels 패딩 (Regression을 위해 float tensor로 처리)
        labels = torch.nn.utils.rnn.pad_sequence(
            labels, batch_first=True, padding_value=0.0  # 🔹 -100 대신 0.0 사용
        ).to(dtype=torch.float32)  # 🔹 float32로 변환

        return dict(
            input_ids=input_ids,
            labels=labels,  # 🔹 Regression을 위한 float labels
            attention_mask=input_ids.ne(self.tokenizer.pad_token_id),
        )


In [14]:
from transformers import DataCollatorWithPadding
from sklearn.model_selection import train_test_split

train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)

print(train_df.shape, eval_df.shape)

train_dataset = SFT_dataset(df=train_df, tokenizer=tokenizer)
eval_dataset = SFT_dataset(df=eval_df, tokenizer=tokenizer)

# 🔹 Data Collator 설정
data_collator = DataCollatorForSupervisedDataset(tokenizer=tokenizer)

(9045, 2) (2262, 2)


In [15]:
import torch

def decode_kogpt2_output(logits, tokenizer):
    """
    KoGPT2의 output(logits)을 디코딩하는 함수.
    logits을 tokenizer를 사용하여 사람이 읽을 수 있는 텍스트로 변환.
    """
    # 🔹 Logits에서 가장 확률이 높은 토큰 선택 (argmax)
    predicted_token_ids = torch.argmax(logits, dim=-1).tolist()  # 🔹 .tolist() 추가

    # 🔹 선택된 토큰을 텍스트로 변환
    decoded_text = tokenizer.decode(predicted_token_ids, skip_special_tokens=True)
    
    return decoded_text

In [16]:
# MSELoss를 사용하도록 Trainer 수정
class RegressionTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, **kargs):
        labels = inputs.pop("labels")  # 정답 라벨 추출
        outputs = model(**inputs)  # 모델 실행
        logits = outputs["logits"]  # 🔹 (batch_size, 5)

        loss_fn = torch.nn.SmoothL1Loss()
        loss = loss_fn(logits, labels)  # 🔹 MSE Loss 적용

        return (loss, outputs) if return_outputs else loss

In [17]:
training_args = TrainingArguments(
    output_dir="./kogpt_sft/test/dt_col/smooth/",
    overwrite_output_dir=True,
    num_train_epochs=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=5,
    prediction_loss_only=False,  # 🔹 softmax가 필요 없으므로 False로 변경
    fp16=True
)

trainer = RegressionTrainer(  # 🔹 RegressionTrainer 사용
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=tokenizer
)

# 모델 학습 실행
trainer.train()



/tmp/ipykernel_3046861/640381666.py:12: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `RegressionTrainer.__init__`. Use `processing_class` instead.
  trainer = RegressionTrainer(  # 🔹 RegressionTrainer 사용


Step,Training Loss
500,0.027200
1000,0.016400
1500,0.014500
2000,0.014400
2500,0.013500
3000,0.013500
3500,0.013400
4000,0.013200
4500,0.014200
5000,0.013100


TrainOutput(global_step=113100, training_loss=0.0038622387886627072, metrics={'train_runtime': 6643.1588, 'train_samples_per_second': 68.078, 'train_steps_per_second': 17.025, 'total_flos': 6.162624047180243e+16, 'train_loss': 0.0038622387886627072, 'epoch': 50.0})

In [18]:
import torch

def predict_ocean(transcription, model, tokenizer):
    """
    KoGPT2 Regression 모델을 사용하여 OCEAN 점수를 예측하는 함수.
    """
    input_text = (
        "### Instruction(명령어):\n주어진 대화를 보고, 성격 5요인을 예측하세요.\n\n"
        f"### Transcription(대화):\n{transcription}\n\n"
        "### Response(응답):"
    )
    
    # Move model to the correct device (GPU if available, otherwise CPU)
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    
    # Tokenize input and move input tensors to the same device
    inputs = tokenizer(input_text, return_tensors="pt").to(device)

    # 모델 실행 (generate() 대신 forward() 호출)
    with torch.no_grad():  # 🔹 Inference 시 Gradient 계산 비활성화
        outputs = model(**inputs)
        logits = outputs["logits"]  # 🔹 OCEAN 값 예측 (batch_size, 5)

    # logits을 0~1 사이 값으로 변환 (Regression Scaling)
    predicted_ocean = torch.sigmoid(logits).squeeze().tolist()  # 🔹 1D 리스트 변환

    return predicted_ocean  # 🔹 5개의 float 값을 리스트로 반환

# ✅ 테스트 실행
sample_transcription = "아."
predicted_ocean = predict_ocean(sample_transcription, model, tokenizer)

print(f"Predicted OCEAN Values: {predicted_ocean}")


Predicted OCEAN Values: [0.6740504503250122, 0.6530162692070007, 0.6811968088150024, 0.6524628400802612, 0.6317079067230225]


In [19]:
import numpy as np
import itertools
from sklearn.metrics import mean_absolute_error
from tqdm import tqdm
from torch.utils.data import DataLoader

device = "cuda" if torch.cuda.is_available() else "cpu"

model.to(device)

# eval_dataset을 DataLoader로 감싸기
eval_dataloader = DataLoader(eval_dataset, batch_size=1)

# 모델을 평가 모드로 설정
model.eval()


KoGPT2ForRegression(
  (transformer): GPT2Model(
    (wte): Embedding(51200, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-11): 12 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (regression_head): Linear(in_features=768, out_features=5, bias=True)
)

In [20]:

# 예측값과 실제값 저장할 리스트 초기화
all_predictions = []
all_labels = []

# 테스트 데이터에 대해 반복문 돌리기
with torch.no_grad():
    for idx, batch in enumerate(tqdm(eval_dataloader)):
        # 데이터 준비
        input_ids = batch['input_ids'].to(device)
        labels_tensor = batch['labels'].to(device)  # 🔹 실제 정답 (OCEAN 값)

        # ✅ 모델 예측 (generate() 대신 forward() 사용)
        outputs = model(input_ids)
        logits = outputs["logits"]  # 🔹 OCEAN 예측값 (batch_size, 5)

         # ✅ 예측값을 0~1 범위로 변환 (sigmoid 사용)
        predicted_ocean = torch.sigmoid(logits).squeeze().tolist()  # 🔹 (batch_size, 5) → 리스트 변환
        # ✅ 실제 정답값 저장
        all_labels.append(labels_tensor.squeeze().tolist())  # 🔹 리스트 변환
        # ✅ 예측값 저장
        all_predictions.append(predicted_ocean)

100%|██████████| 2262/2262 [00:18<00:00, 123.54it/s]


In [21]:
print(all_predictions)


[[0.6700687408447266, 0.641983687877655, 0.6539008617401123, 0.6563279032707214, 0.6214834451675415], [0.6695287823677063, 0.6409730315208435, 0.6676896214485168, 0.6567683815956116, 0.6109758615493774], [0.6698528528213501, 0.6386099457740784, 0.6568784117698669, 0.6557770371437073, 0.6171088814735413], [0.6715782880783081, 0.6444488763809204, 0.6548947095870972, 0.6547843813896179, 0.6188956499099731], [0.6556668281555176, 0.6455669403076172, 0.6671476364135742, 0.6531268954277039, 0.5930790901184082], [0.6699607968330383, 0.6417592167854309, 0.6504672765731812, 0.6520198583602905, 0.6218854188919067], [0.6721165180206299, 0.6428809762001038, 0.6493563055992126, 0.6530162692070007, 0.6226887702941895], [0.6676896214485168, 0.6411977410316467, 0.6555565595626831, 0.6605009436607361, 0.6223445534706116], [0.6653018593788147, 0.6441131830215454, 0.6561076641082764, 0.6536798477172852, 0.6045740246772766], [0.6694207787513733, 0.6383845806121826, 0.6569884419441223, 0.6598436236381531, 0

In [22]:
print(all_labels)

[[0.6875, 0.4791666567325592, 0.4375, 0.7708333134651184, 0.4375], [0.5625, 0.8333333134651184, 0.8125, 0.3125, 0.2708333432674408], [0.4375, 0.7083333134651184, 0.5625, 0.3958333432674408, 0.4166666567325592], [0.7291666865348816, 0.4791666567325592, 0.5, 0.4583333432674408, 0.7291666865348816], [0.6041666865348816, 0.875, 0.8333333134651184, 0.5, 0.625], [0.7708333134651184, 0.7083333134651184, 0.6666666865348816, 0.5208333134651184, 0.5416666865348816], [0.4166666567325592, 0.5416666865348816, 0.5416666865348816, 0.5, 0.6875], [0.5625, 0.8333333134651184, 0.8125, 0.3125, 0.2708333432674408], [0.6458333134651184, 0.5833333134651184, 0.8541666865348816, 0.5833333134651184, 0.2708333432674408], [0.9791666865348816, 0.9791666865348816, 0.8333333134651184, 0.6458333134651184, 0.2083333283662796], [0.5625, 0.8333333134651184, 0.8125, 0.3125, 0.2708333432674408], [0.5416666865348816, 0.5625, 0.8125, 0.5833333134651184, 0.2916666567325592], [0.8333333134651184, 0.6041666865348816, 0.5625, 0

In [23]:
import numpy as np

# 🔹 numpy 배열 변환
labels_array = np.array(all_labels)
predictions_array = np.array(all_predictions)

# 🔹 소수점 4자리까지 반올림
labels_rounded = np.round(labels_array, 4)
predictions_rounded = np.round(predictions_array, 4)

# ✅ 출력
print("🔹 Rounded Predictions:")
print(predictions_rounded.tolist())  # 리스트 형태로 변환하여 출력

print("\n🔹 Rounded Labels:")
print(labels_rounded.tolist())  # 리스트 형태로 변환하여 출력


🔹 Rounded Predictions:
[[0.6701, 0.642, 0.6539, 0.6563, 0.6215], [0.6695, 0.641, 0.6677, 0.6568, 0.611], [0.6699, 0.6386, 0.6569, 0.6558, 0.6171], [0.6716, 0.6444, 0.6549, 0.6548, 0.6189], [0.6557, 0.6456, 0.6671, 0.6531, 0.5931], [0.67, 0.6418, 0.6505, 0.652, 0.6219], [0.6721, 0.6429, 0.6494, 0.653, 0.6227], [0.6677, 0.6412, 0.6556, 0.6605, 0.6223], [0.6653, 0.6441, 0.6561, 0.6537, 0.6046], [0.6694, 0.6384, 0.657, 0.6598, 0.6204], [0.6681, 0.6437, 0.6516, 0.65, 0.6157], [0.666, 0.6424, 0.6568, 0.6618, 0.6175], [0.6621, 0.6449, 0.6616, 0.6581, 0.6053], [0.6671, 0.6415, 0.6573, 0.6593, 0.6198], [0.6673, 0.6423, 0.6558, 0.65, 0.6164], [0.6615, 0.6383, 0.6612, 0.6562, 0.6111], [0.6731, 0.6441, 0.6542, 0.6597, 0.6244], [0.6696, 0.6395, 0.6549, 0.659, 0.6226], [0.6705, 0.6377, 0.655, 0.6608, 0.6225], [0.6696, 0.6392, 0.6584, 0.659, 0.6195], [0.6661, 0.641, 0.6564, 0.6589, 0.6201], [0.669, 0.6389, 0.6538, 0.6569, 0.6237], [0.666, 0.6402, 0.658, 0.6519, 0.617], [0.6709, 0.6423, 0.6542, 0.6614

In [24]:
# 🔹 MAE 계산
mae_score = mean_absolute_error(labels_rounded, predictions_rounded)

In [25]:
# 🔹 1-MAE 출력
one_minus_mae = 1 - mae_score
print(f"✅ 1-MAE Score: {one_minus_mae}")

✅ 1-MAE Score: 0.8528388152077808


In [26]:
# 🔹 각 OCEAN 요소별 MAE 계산
mae_per_trait = []
for i in range(5):
    mae = mean_absolute_error(labels_array[:, i], predictions_array[:, i])  # i번째 요소에 대해 MAE 계산
    mae_per_trait.append(mae)

In [27]:
# 🔹 1 - MAE 계산
one_minus_mae_per_trait = [1 - mae for mae in mae_per_trait]

# ✅ 결과 출력
trait_names = ["O", "C", "E", "A", "N"]
for name, score in zip(trait_names, one_minus_mae_per_trait):
    print(f"1-MAE for {name}: {score:.4f}")

# ✅ 평균 1-MAE 출력
mean_one_minus_mae = np.mean(one_minus_mae_per_trait)
print(f"✅ Mean 1-MAE Score: {mean_one_minus_mae:.4f}")

1-MAE for O: 0.8709
1-MAE for C: 0.8821
1-MAE for E: 0.8631
1-MAE for A: 0.8672
1-MAE for N: 0.7809
✅ Mean 1-MAE Score: 0.8528


In [28]:
# ✅ 테스트 실행
sample_transcription = "아 그럼 3시 반."
predicted_ocean = predict_ocean(sample_transcription, model, tokenizer)

print(f"Predicted OCEAN Values: {predicted_ocean}")

Predicted OCEAN Values: [0.6697448492050171, 0.6647579669952393, 0.6557770371437073, 0.658307671546936, 0.6309123635292053]
